## Step 1️: Set up your OpenAI or Azure OpenAI Service key

In [ ]:
#!import ../config/Settings.cs

bool useAzureOpenAI = false;

await Settings.AskAzureEndpoint(useAzureOpenAI);
await Settings.AskModel(useAzureOpenAI);
await Settings.AskApiKey(useAzureOpenAI);

## Step 2: Instantiating a kernel

In [ ]:
#r "nuget: Microsoft.SemanticKernel, 0.9.61.1-preview"

#!import ../config/Settings.cs

using Microsoft.SemanticKernel;
using Microsoft.SemanticKernel.KernelExtensions;
using System.IO;
using Microsoft.SemanticKernel.Configuration;
using Microsoft.SemanticKernel.SemanticFunctions;

IKernel kernel = Microsoft.SemanticKernel.Kernel.Builder.Build();

// Grab the locally stored credentials from the settings.json file. Name the "backend" as "davinci" — assuming that you're using one of the davinci completion models. 

var (useAzureOpenAI, model, azureEndpoint, apiKey, orgId) = Settings.LoadFromFile();

if (useAzureOpenAI)
    kernel.Config.AddAzureOpenAITextCompletion("davinci", model, azureEndpoint, apiKey);
else
    kernel.Config.AddOpenAITextCompletion("davinci", model, apiKey, orgId);

## Step 3: Running our code

In [ ]:
using Microsoft.SemanticKernel.Orchestration;
using Microsoft.DotNet.Interactive;
using System;
using System.Net.Http;
using System.Threading.Tasks;
using System.Collections.Generic;
using Newtonsoft.Json;

// Setting Http Client
HttpClient client = new HttpClient();
client.DefaultRequestHeaders.Accept.Clear();
client.DefaultRequestHeaders.Accept.Add(
new System.Net.Http.Headers.MediaTypeWithQualityHeaderValue("application/json"));

// retrieve Customer data
var response = await client.GetAsync("https://sk-api-backend.azurewebsites.net/api/customer/getInsuranceCustomers");
var customerData = await response.Content.ReadAsStringAsync();

// retrieve certification data
response = await client.GetAsync("https://sk-api-backend.azurewebsites.net/api/customer/getDrivingCertificates");
var certificationData = await response.Content.ReadAsStringAsync();

// Load the Skills Directory
var skillsDirectory = Path.Combine(System.IO.Directory.GetCurrentDirectory(), "skills");

// Load the ChatSkill from the Skills Directory
var mySkill = kernel.ImportSemanticSkillFromDirectory(skillsDirectory, "ChatSkill");


var myInput = await Microsoft.DotNet.Interactive.Kernel.GetInputAsync();
var CustInfo = new ContextVariables();
CustInfo.Set("customerData", customerData.ToString());
CustInfo.Set("certificationData", certificationData.ToString());
CustInfo.Set("input", myInput.ToString());
CustInfo.Set("history", "");

// Run the Function called Joke with the default parameter of $input
int maxLimit = 20;
int currentInteraction = 0;
while(currentInteraction < maxLimit) {
  var result = await kernel.RunAsync(CustInfo, mySkill["Chat"]);
  Console.WriteLine(result);
  
  var historychat = "me: " + myInput + "\n AI: " + result;
  CustInfo.Set("history", historychat);
  myInput = await Microsoft.DotNet.Interactive.Kernel.GetInputAsync();
  CustInfo.Set("input", myInput.ToString());
  currentInteraction++;
}